In [112]:
import math
import numpy as np
import pandas as pd

In [113]:
def get_probability(event_info):
  SUM = sum(event_info)
  for i in range(len(event_info)):
        event_info[i] /= SUM
  return event_info


def get_entropy(event_info):
  probabilities = get_probability(event_info)
  entropy = 0.0
  for p in probabilities:
    if p != 0:
      entropy += p * math.log(1 / p) / math.log(2)
  return entropy


In [114]:
def sort_table_by_column(table, col):
    return table[table[:, col].argsort()]

In [115]:
# for importing the dataset as a numpy array
dataset1 = pd.read_csv(r"covid_dataset.csv")
dataset1 = dataset1[dataset1['location'] == 'India']
# dataset1 = dataset1.to_numpy()
dataset2 = pd.read_csv(r"changes-visitors-covid.csv")
dataset2 = dataset2[dataset2['Entity'] == 'India']
# dataset2 = dataset2.to_numpy()

In [116]:
dataset = pd.merge(dataset1, dataset2, on = 'date', how = 'inner')

In [117]:
example_dataset = dataset1 = pd.read_csv(r"example_data.csv")

In [118]:
print(example_dataset)

  Outlook Company Sailboat target
0   sunny     big    small    yes
1   sunny     med    small    yes
2   sunny     med      big    yes
3   sunny      no    small    yes
4   sunny     big      big    yes
5   rainy      no    small     no
6   rainy     med    small    yes
7   rainy     big      big    yes
8   rainy      no      big     no
9   rainy     med      big     no


In [119]:
def split_dataset_wrt_column(dataset, column_name):
  unique_items = dataset[column_name].unique()
  for item in unique_items:
    yield dataset[dataset[column_name] == item]

In [120]:
class node:
  def __init__(self):
    self.table = None
    self.spliting_feature = None
    self.childs = []
  def add_child(self, child):
    self.childs.append(child)

In [121]:
def get_count(table, target_column, class_name):
  # print(class_name)
  # ans =  (table[target_column] == class_name).shape[0]
  ans = (table[target_column] == class_name).sum()
  # print (ans)
  return ans

In [122]:
def get_entropy_from_table(table, target_column):
  unique_classes = table[target_column].unique()
  # print(unique_classes)
  counts = []
  for class_name in unique_classes:
    counts.append(get_count(table,target_column, class_name))
  # print("Count is ", counts)
  return get_entropy(counts)
  

In [123]:
def get_information_gain(dataset, column, target_column):
  tables = []
  size_table = []
  overall_size = dataset.shape[0]
  for table in split_dataset_wrt_column(dataset, column):
    tables.append(table)
    size_table.append(table.shape[0])
  entropies = []
  for table in tables:
    # print(table)
    entropies.append(get_entropy_from_table(table, target_column))
  print(entropies)
  # entropies = [get_entropy_from_table(table, target_column) for table in tables]
  # print(entropies)
  # entropy = sum([(size / overall_size) * entropyi for size, entropyi in zip(sizes, entropies)])
  # return entropy


In [124]:
get_information_gain(example_dataset, 'Sailboat', 'target')

[0.7219280948873624, 0.9709505944546688]
